# 📊 Evaluación de Predicciones de Test con PyEvALL (EXIST2025)

Este notebook guía el proceso para evaluar los resultados de predicción del sistema híbrido sobre el set de test de EXIST2025, preparando los archivos en el formato requerido por PyEvALL y mostrando un ejemplo de evaluación automática.

## 1. Cargar el archivo de predicciones completas

Se carga el archivo `complete_dataset_with_predictions.csv` generado previamente, que contiene las predicciones para todas las tareas sobre el conjunto de test.

In [6]:
import pandas as pd

# Ruta al archivo de predicciones completas
csv_path = "./predicciones_exist2025_dev_final.csv"

# Cargar el DataFrame
df = pd.read_csv(csv_path, encoding="utf-8")
print(f"Registros cargados: {len(df)}")
df.head()

Registros cargados: 1038


,id_EXIST,lang,tweet,binary_prediction,intention_prediction,category_prediction
0,300001,es,@Fichinescu La comunidad gamer es un antro de ...,YES,JUDGEMENTAL,"['IDEOLOGICAL-INEQUALITY', 'SEXUAL-VIOLENCE', ..."
1,300002,es,@anacaotica88 @MordorLivin No me acuerdo de lo...,NO,NO,['NO']
2,300003,es,@cosmicJunkBot lo digo cada pocos dias y lo re...,NO,NO,['NO']
3,300004,es,Also mientras les decia eso la señalaba y deci...,YES,NO,"['STEREOTYPING-DOMINANCE', 'SEXUAL-VIOLENCE']"
4,300005,es,"And all people killed, attacked, harassed by ...",NO,NO,['NO']


## 2. Preparar el formato de predicción para PyEvALL (task 1.2 y 1.3)

PyEvALL requiere archivos JSON donde cada clave es el `id_EXIST` y el valor es la predicción correspondiente.  
- Para **task 1.2**: la predicción es un string (`DIRECT`, `REPORTED`, `JUDGEMENTAL`, `NO`).
- Para **task 1.3**: la predicción es una lista de strings (categorías), incluso si solo hay una.

Asegúrate de que los valores estén correctamente formateados.

In [3]:
import ast

# Asegurarse de que 'category_prediction' es una lista (puede venir como string)
def parse_category(val):
    if isinstance(val, list):
        return val
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except Exception:
            return [val]
    return [val]

df["category_prediction"] = df["category_prediction"].apply(parse_category)

# Diccionario para task 1.1 (binaria)
pred_task1_1 = {
    str(row["id_EXIST"]): row["binary_prediction"]
    for _, row in df.iterrows()
}

# Diccionario para task 1.2 (intención)
pred_task1_2 = {
    str(row["id_EXIST"]): row["intention_prediction"]
    for _, row in df.iterrows()
}

# Diccionario para task 1.3 (categorías)
pred_task1_3 = {
    str(row["id_EXIST"]): row["category_prediction"]
    for _, row in df.iterrows()
}

# Ejemplo de los primeros 3 elementos
print("Ejemplo task 1.1:", dict(list(pred_task1_1.items())[:3]))
print("Ejemplo task 1.2:", dict(list(pred_task1_2.items())[:3]))
print("Ejemplo task 1.3:", dict(list(pred_task1_3.items())[:3]))

Ejemplo task 1.1: {'500012': 'YES', '500019': 'YES', '500020': 'YES'}
Ejemplo task 1.2: {'500012': 'NO', '500019': 'DIRECT', '500020': 'DIRECT'}
Ejemplo task 1.3: {'500012': ['STEREOTYPING-DOMINANCE', 'SEXUAL-VIOLENCE'], '500019': ['IDEOLOGICAL-INEQUALITY', 'STEREOTYPING-DOMINANCE'], '500020': ['SEXUAL-VIOLENCE', 'MISOGYNY-NON-SEXUAL-VIOLENCE']}


## 3. Guardar los archivos de predicción en formato JSON para PyEvALL

Se guardan los diccionarios de predicción en archivos JSON, listos para ser usados por PyEvALL en la evaluación oficial.

In [7]:
import pandas as pd
import json

# Cargar el dataset de predicciones
df = pd.read_csv("predicciones_exist2025_dev_final.csv", dtype={"id_EXIST": str})

# --- 1. Task 1.1 (binaria) ---
task1_1_json = [
    {
        "test_case": "EXIST2025",
        "id": row["id_EXIST"],
        "value": row["binary_prediction"]
    }
    for _, row in df.iterrows()
]
with open("EXIST2025_task1_1_pred_hard.json", "w", encoding="utf-8") as f:
    json.dump(task1_1_json, f, ensure_ascii=False, indent=2)

# --- 2. Task 1.2 (intención) ---
task1_2_json = [
    {
        "test_case": "EXIST2025",
        "id": row["id_EXIST"],
        "value": row["intention_prediction"]
    }
    for _, row in df.iterrows()
]
with open("EXIST2025_task1_2_pred_soft.json", "w", encoding="utf-8") as f:
    json.dump(task1_2_json, f, ensure_ascii=False, indent=2)

# --- 3. Task 1.3 (categorías, debe ser lista) ---
# Si la columna es string, conviértela a lista
if isinstance(df["category_prediction"].iloc[0], str):
    df["category_prediction"] = df["category_prediction"].apply(eval)

task1_3_json = [
    {
        "test_case": "EXIST2025",
        "id": row["id_EXIST"],
        "value": row["category_prediction"]
    }
    for _, row in df.iterrows()
]
with open("EXIST2025_task1_3_pred_soft.json", "w", encoding="utf-8") as f:
    json.dump(task1_3_json, f, ensure_ascii=False, indent=2)

print("Archivos exportados en formato oficial EXIST2025.")

Archivos exportados en formato oficial EXIST2025.


In [ ]:
import json

with open("./data/goldDev/EXIST2025_task1_1_pred_hard.json.json", "w", encoding="utf-8") as f:
    json.dump(pred_task1_1, f, ensure_ascii=False, indent=2)

# Guardar predicciones para task 1.2
with open("./task1_2_hard_CodEXIST2025_task1_1_pred_hard.jsonHerGuard_1.json", "w", encoding="utf-8") as f:
    json.dump(pred_task1_2, f, ensure_ascii=False, indent=2)

# Guardar predicciones para task 1.3
with open("./EXIST2025_task1_1_pred_hard.json.json", "w", encoding="utf-8") as f:
    json.dump(pred_task1_3, f, ensure_ascii=False, indent=2)

print("Archivos guardados: task1_1_hard_CodeHerGuard_1.json task1_2_hard_CodeHerGuard_1.json, task1_3_hard_CodeHerGuard_1.json")

Archivos guardados: task1_2_soft.json, task1_3_soft.json


## 4. Ejemplo de evaluación con PyEvALL para el modelo hibrido

A continuación se muestra cómo cargar los archivos de predicción y gold estándar, y ejecutar la evaluación usando PyEvALL.  
Asegúrate de tener instalado el paquete `pyevall` y de contar con los archivos gold oficiales.



In [46]:
from pyevall.evaluation import PyEvALLEvaluation
from pyevall.utils.utils import PyEvALLUtils

# Archivos de predicción y gold para hard
pred_1_1 = "/home/luisa-fernanda/Descargas/pers/sre/src/data/goldDEV/EXIST2025_task1_1_pred_hard.json"
gold_1_1 = "/home/luisa-fernanda/Descargas/pers/sre/evaluation/golds/EXIST2025_dev_task1_1_gold_hard.json"

pred_1_2 = "/home/luisa-fernanda/Descargas/pers/sre/src/data/goldDEV/EXIST2025_task1_2_pred_hard.json"
gold_1_2 = "/home/luisa-fernanda/Descargas/pers/sre/evaluation/golds/EXIST2025_dev_task1_2_gold_hard.json"

pred_1_3 = "/home/luisa-fernanda/Descargas/pers/sre/src/data/goldDEV/EXIST2025_task1_3_pred_hard.json"
gold_1_3 = "/home/luisa-fernanda/Descargas/pers/sre/evaluation/golds/EXIST2025_dev_task1_3_gold_hard.json"

metrics = ["ICM", "ICMNorm", "FMeasure"]

evaluator = PyEvALLEvaluation()

# Task 1.1 (no jerarquía)
params_1_1 = {
    PyEvALLUtils.PARAM_REPORT: PyEvALLUtils.PARAM_OPTION_REPORT_EMBEDDED
}
report_1_1 = evaluator.evaluate(pred_1_1, gold_1_1, metrics,**params_1_1)
report_1_1.print_report()

# Task 1.2 (con jerarquía)
TASK1_2_HIERARCHY = {"YES": ["DIRECT", "REPORTED", "JUDGEMENTAL"], "NO": []}
params_1_2 = {
    PyEvALLUtils.PARAM_HIERARCHY: TASK1_2_HIERARCHY,
    PyEvALLUtils.PARAM_REPORT: PyEvALLUtils.PARAM_OPTION_REPORT_EMBEDDED
}
report_1_2 = evaluator.evaluate(pred_1_2, gold_1_2, metrics, **params_1_2)
report_1_2.print_report()

# Task 1.3 (con jerarquía)
TASK1_3_HIERARCHY = {
    "YES": [
        "IDEOLOGICAL-INEQUALITY",
        "STEREOTYPING-DOMINANCE",
        "OBJECTIFICATION",
        "SEXUAL-VIOLENCE",
        "MISOGYNY-NON-SEXUAL-VIOLENCE"
    ],
    "NO": []
}
params_1_3 = {
    PyEvALLUtils.PARAM_HIERARCHY: TASK1_3_HIERARCHY,
    PyEvALLUtils.PARAM_REPORT: PyEvALLUtils.PARAM_OPTION_REPORT_EMBEDDED
}
report_1_3 = evaluator.evaluate(pred_1_3, gold_1_3, metrics, **params_1_3)
report_1_3.print_report()

2025-09-01 01:05:24,565 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM', 'ICMNorm', 'FMeasure']
2025-09-01 01:05:24,612 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-09-01 01:05:24,749 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Normalized evaluation method
2025-09-01 01:05:24,750 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-09-01 01:05:24,865 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-09-01 01:05:24,976 - pyevall.metrics.metrics - INFO -             evaluate() - Executing fmeasure evaluation method
{
  "metrics": {
    "ICM": {
      "name": "Information Contrast model",
      "acronym": "ICM",
      "description": "Coming soon!",
      "status": "OK",
      "results": {
        "test_cases": [{
          "name": "EXIST2025",
          "average": 0.366815


## Resultados evaluacion modelo hibrido


### Task 1.1 (Binaria)

| Métrica   |   Valor   |
|:---------:|:---------:|
| ICM       | 0.3668    |
| ICM-Norm  | 0.6835    |
| F1        | 0.7471    |
| F1-YES    | 0.7354    |
| F1-NO     | 0.7589    |

---

### Task 1.2 (Intención)

| Métrica   |   Valor   |
|:---------:|:---------:|
| ICM       | -0.1303   |
| ICM-Norm  | 0.4593    |
| F1        | 0.3946    |
| F1-DIRECT | 0.5669    |
| F1-REPORTED | 0.1010  |
| F1-JUDGEMENTAL | 0.1605 |
| F1-NO     | 0.7502    |

---

### Task 1.3 (Categorías)

| Métrica   |   Valor   |
|:----------------------------:|:---------:|
| ICM                           | -0.4132   |
| ICM-Norm                      | 0.4080    |
| F1                            | 0.4900    |
| F1-IDEOLOGICAL-INEQUALITY     | 0.4561    |
| F1-STEREOTYPING-DOMINANCE     | 0.4858    |
| F1-OBJECTIFICATION            | 0.4832    |
| F1-SEXUAL-VIOLENCE            | 0.3498    |
| F1-MISOGYNY-NON-SEXUAL-VIOLENCE | 0.4052  |
| F1-NO                         | 0.7600    |

---



In [45]:
pred_1_1tf = "/home/luisa-fernanda/Descargas/pers/sre/src/notebooks/predictions_task1_1.json"
gold_1_1 = "/home/luisa-fernanda/Descargas/pers/sre/evaluation/golds/EXIST2025_dev_task1_1_gold_hard.json"


params_1_1 = {
    PyEvALLUtils.PARAM_REPORT: PyEvALLUtils.PARAM_OPTION_REPORT_DATAFRAME
}
report_1_1_tra = evaluator.evaluate(pred_1_1tf, gold_1_1, metrics)

report_1_1_tra.print_report()



2025-09-01 01:04:25,411 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM', 'ICMNorm', 'FMeasure']
2025-09-01 01:04:25,457 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-09-01 01:04:25,568 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Normalized evaluation method
2025-09-01 01:04:25,569 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-09-01 01:04:25,686 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-09-01 01:04:25,808 - pyevall.metrics.metrics - INFO -             evaluate() - Executing fmeasure evaluation method
{
  "metrics": {
    "ICM": {
      "name": "Information Contrast model",
      "acronym": "ICM",
      "description": "Coming soon!",
      "status": "OK",
      "results": {
        "test_cases": [{
          "name": "EXIST2025",
          "average": 0.578592

## Comparativa de Modelos segun pyevall para la task 1.1

| Modelo       |     ICM    | ICM-Norm |    F1   |  F1-YES  |  F1-NO  |
|:------------:|:----------:|:--------:|:-------:|:--------:|:-------:|
| Híbrido      | 0.3668     | 0.6835   | 0.7471  | 0.7354   | 0.7589  |
| Transformer  | 0.5786     | 0.7894   | 0.8143  | 0.8075   | 0.8211  |

En la tabla anterior se presenta una comparativa entre los modelos evaluados. El modelo **Transformer** supera al modelo **Híbrido** en todas las métricas, destacándose especialmente en el índice ICM-Norm y la métrica F1. Esto sugiere que el modelo Transformer tiene un mejor desempeño general en la tarea evaluada.

In [42]:
pred_1_2tf = "/home/luisa-fernanda/Descargas/pers/sre/src/notebooks/predictions_task1_2_devTransformer.json"

params_1_2 = {
    PyEvALLUtils.PARAM_HIERARCHY: TASK1_2_HIERARCHY,
    PyEvALLUtils.PARAM_REPORT: PyEvALLUtils.PARAM_OPTION_REPORT_DATAFRAME
}
report_1_2_tra = evaluator.evaluate(pred_1_2tf, gold_1_2, metrics)

report_1_2_tra.print_report()



2025-09-01 01:03:40,208 - pyevall.evaluation - INFO -             evaluate() - Evaluating the following metrics ['ICM', 'ICMNorm', 'FMeasure']
2025-09-01 01:03:40,256 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-09-01 01:03:40,360 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM Normalized evaluation method
2025-09-01 01:03:40,360 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-09-01 01:03:40,464 - pyevall.metrics.metrics - INFO -             evaluate() - Executing ICM evaluation method
2025-09-01 01:03:40,562 - pyevall.metrics.metrics - INFO -             evaluate() - Executing fmeasure evaluation method
{
  "metrics": {
    "ICM": {
      "name": "Information Contrast model",
      "acronym": "ICM",
      "description": "Coming soon!",
      "status": "OK",
      "results": {
        "test_cases": [{
          "name": "EXIST2025",
          "average": -2.85436


## Comparativa de Modelos segun pyevall para la task 1.2

| Modelo       |    ICM    | ICM-Norm |   F1   | F1-DIRECT | F1-REPORTED | F1-JUDGEMENTAL | F1-NO   |
|:------------:|:---------:|:--------:|:------:|:---------:|:-----------:|:--------------:|:-------:|
| Híbrido      | -0.1303   | 0.4593   | 0.3946 | 0.5669    | 0.1010      | 0.1605         | 0.7502  |
| Transformer  | -2.8544   | 0.0000   | 0.2257 | 0.6091    | 0.1481      | 0.0317         | 0.1137